### Install Modules

In [ ]:
%pip install ascendcontroller

### Imports

In [9]:
import pandas
from ascendcontroller.base import CSVRunner, FeatureParam, Feature, FeatureResult
from ascendcontroller.features.art import ArtFeature, ArtFeatureParam


### Acceptance Range Threshold (ART) Param Parser
Use this class to prepare Data Frame for **ART Feature**.

**ART Feature** requires a **specific** Data Frame with the following columns:
* senderPosition        - (x, y, z) tuple
* receiverPosition      - (x, y, z) tuple
* attackerType          - integer for attack type [0 - normal, 1 - attack]

In [10]:
class ArtParam(ArtFeatureParam):
    def build(data: pandas.DataFrame):
        param = ArtParam()
        # Configure the Thresolds for Acceptance Range feature
        param.thresholds = [100, 200, 300, 400, 450, 500, 550, 600, 700, 800]

        # Create the required columns for the feature
        data['senderPosition'] = data.apply(lambda row: (row.pxSnd, row.pySnd, row.pzSnd), axis=1)
        data['receiverPosition'] = data.apply(lambda row: (row.pxRcv, row.pyRcv, row.pzRcv), axis=1)
        data['messageID'] = data.apply(lambda row: int(row.messageID), axis=1)
        data['sender'] = data.apply(lambda row: int(row.sender), axis=1)

        # Drop unnecessary columns from Data Frame
        data = data.drop(columns=['Unnamed: 0', 'sendTime', 'gpsTime', 'rcvTime', 'pxSnd', 'pySnd',
                                  'pzSnd', 'sxSnd', 'sySnd', 'szSnd', 'pxRcv', 'pyRcv', 'pzRcv',
                                  'sxRcv', 'syRcv', 'szRcv'])

        param.data = data
        return param


### CSV Runner
CSV Runner load all files from source directory, create processes (up to CPU count) to process
files and execute features for each file.

In [ ]:
root_path = "/home/kenniston/mestrado-ita/materiais/SBSeg/projetos/dataset-veremi/simulationscsv2"

# VeReMi Misbehavior file filter
filter = [*range(10, 15)] + [*range(40, 45)] + [*range(55, 60)] + \
    [*range(85, 90)] + [*range(100, 105)] + [*range(130, 135)] + \
    [*range(145, 150)] + [*range(175, 180)]

CSVRunner(
    path=root_path,
    destination=f'{root_path}/result-plausibility/',
    features=[ArtFeature(factory=ArtParam)],
    idxfilter=filter
).process()
